# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [7]:
from gtts import gTTS 
import time
import numpy as np
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import cv2
cap = cv2.VideoCapture(0)

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


ImportError: Traceback (most recent call last):
  File "C:\Users\Client\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\Client\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\Client\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\Client\Anaconda3\lib\imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\Client\Anaconda3\lib\imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Le module spécifié est introuvable.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for

## Env setup

In [3]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [2]:
from utils import label_map_util

from utils import visualization_utils as vis_util

ModuleNotFoundError: No module named 'tensorflow'

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [5]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

## Download Model

In [6]:
#opener = urllib.request.URLopener()
#opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
#tar_file = tarfile.open(MODEL_FILE)
#for file in tar_file.getmembers():
  #file_name = os.path.basename(file.name)
  #if 'frozen_inference_graph.pb' in file_name:
    #tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [7]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [8]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [10]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    while True:
      threshold=0.5
      ret, image_np = cap.read()
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
      scores = detection_graph.get_tensor_by_name('detection_scores:0')
      classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
      # Actual detection.
      (boxes, scores, classes, num_detections) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)

      cv2.imshow('object detection', cv2.resize(image_np, (800,600)))
     ''' objects = []
      for index, value in enumerate(classes[0]):
        object_dict = {}
        if scores[0, index] > threshold:
              object_dict[(category_index.get(value)).get('name').encode('utf8')] = \
                        scores[0, index]
              objects.append(object_dict)
              print((category_index.get(value)).get('name'))  
              if(str((category_index.get(value)).get('name'))=="person"):                  
                #language = 'ar'
                #myobj = gTTS(text=mytext, lang=language, slow=False)
                #myobj.save("person.mp3")
                #os.system("start C:/Users/Hola/Desktop/Nouveau/person.mp3")
                #time.sleep(5)
                #os.system("python C:/Users/Hola/Desktop/Ttensorflow/models/research/object_detection/Face/face_recognition.py")
                print("hi")
                #time.sleep(7)
              elif(str((category_index.get(value)).get('name'))=="traffic light"):
                #mytext ='لقد تم رصد إشارة مرور ، سيتم التثبت'  
                #language = 'ar'
                #myobj = gTTS(text=mytext, lang=language, slow=False)
                #myobj.save("traffic.mp3")
                os.system("start C:/Users/Hola/Desktop/Nouveau/traffic.mp3")
                time.sleep(5)
                os.system("python C:/Users/Hola/Desktop/traf.py ")
              elif(str((category_index.get(value)).get('name'))=="car"):  
                #mytext ='إحذر  هناك سيارة'  
                #language = 'ar'
                #myobj = gTTS(text=mytext, lang=language, slow=False)
                #myobj.save("car.mp3")
                os.system("start C:/Users/Hola/Desktop/Nouveau/car.mp3")
                time.sleep(5)'''
      if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

person
hi
person
hi
person
hi
person
hi
person
hi
dog
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person
hi
person

# Detection